# O output desse processamento deve ser um arquivo com schema pronto e juntando todos os arquivos feitos no passo 1

##1. Importando as bibliotecas

In [ ]:
import subprocess
import pandas as pd

##2. Extraindo os arquivos gerados no passo 1

In [ ]:
subprocess.run(['unrar','x','/content/drive/MyDrive/dados.rar'])

CompletedProcess(args=['unrar', 'x', '/content/drive/MyDrive/dados.rar'], returncode=0)

##3. Criando pastas de apoio

In [ ]:
subprocess.run(['mkdir','/content/Passo1'])
subprocess.run(['mkdir','/content/Passo2'])
subprocess.run(['mkdir','/content/Passo3'])
subprocess.run(['mkdir','/content/Passo4'])

CompletedProcess(args=['mkdir', '/content/Passo4'], returncode=0)

##4. Criando uma lista com o nome dos arquivos

In [ ]:
files = subprocess.getoutput(['ls','/content/'])
files = files.split()
files.pop()
files.pop()
files.pop()
files.pop()
files.pop()
files.pop()

'drive'

##5. Transformando os dados numéricos

In [ ]:
for fi in files:
  fin = open(fi, "rt")
  fout = open('/content/Passo1/'+fi, "wt")
  for line in fin:
	    fout.write(line.replace(',', '.'))
fin.close()
fout.close()

In [ ]:
for fi in files:
  fin = open('/content/Passo1/'+fi, "rt")
  fout = open('/content/Passo2/'+fi, "wt")
  for line in fin:
	    fout.write(line.replace(';', ','))
fin.close()
fout.close()

##6. Cria novos arquivos com pressão menor que 5

In [ ]:
for fi in files:
  df = pd.read_csv('/content/Passo2/'+fi)
  df2 = df.loc[df['PRES (decibar)']<5]
  df2.to_csv('/content/Passo3/'+fi)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


##7. Cria novos arquivos preparados como variaveis

In [ ]:
for fi in files:  
  df = pd.read_csv('/content/Passo3/'+fi,parse_dates=[2])
  df['ANO'] = df['DATE (YYYY-MM-DDTHH:MI:SSZ)'].dt.year
  df['MES'] = df['DATE (YYYY-MM-DDTHH:MI:SSZ)'].dt.month
  df['DIA'] = df['DATE (YYYY-MM-DDTHH:MI:SSZ)'].dt.day
  df['LATITUDE'] = df['LATITUDE (degree_north)']
  df['LONGITUDE'] = df['LONGITUDE (degree_east)']
  df['SALINIDADE'] = df['PSAL (psu)']
  df['PRESS'] = df['PRES (decibar)']
  dx=df[['ANO','MES','DIA','LATITUDE','LONGITUDE','SALINIDADE','PRESS']]
  dx.to_csv('/content/Passo4/'+fi)

##8. Criando um unico ponto de dados sem dados nulos

In [ ]:
dfFinal = pd.DataFrame(data=None)
for fi in files:
  df = pd.read_csv('/content/Passo4/'+fi) 
  df.dropna(inplace=True)
  df.drop_duplicates(inplace=True)
  dfFinal = pd.concat([dfFinal,df])

##9. Salvando o df 

In [ ]:
dfFinal = dfFinal[['ANO','MES','DIA','LATITUDE','LONGITUDE','SALINIDADE','PRESS']] 
dfFinal.to_csv('/content/drive/MyDrive/dados.csv')